#Data Import 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
pip install xgboost

#Run k-fold CV 


In [ ]:
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier


X = data.drop(['target'], axis=1)
y = data.target
cv = KFold(n_splits=10, random_state=1, shuffle=True)
cv1 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.5)

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

SVM = SVC(kernel='linear', class_weight='balanced')#using balanced svm
RFC = RandomForestClassifier()
XGB = XGBClassifier()
MLP = MLPClassifier()

scores = cross_val_score(SVM, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
#scores1 = cross_val_score(SVM, X, y, scoring='accuracy', cv=cv1, n_jobs=-1)
print('SVM Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))
#print('SVM Accuracy Repeated 10 fold: %.3f (%.3f)' % (mean(scores1), std(scores1)))
scores = cross_val_score(RFC, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
#scores1 = cross_val_score(RFC, X, y, scoring='accuracy', cv=cv1, n_jobs=-1)
print('RFC Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))
#print('RFC Accuracy Repeated 10 fold: %.3f (%.3f)' % (mean(scores1), std(scores1)))
scores = cross_val_score(XGB, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
#scores1 = cross_val_score(XGB, X, y, scoring='accuracy', cv=cv1, n_jobs=-1)
print('XGB Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))
#print('XGB Accuracy Repeated 10 fold: %.3f (%.3f)' % (mean(scores1), std(scores1)))
scores = cross_val_score(MLP, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('MLP Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))


SVM Accuracy 10 fold: 81.8817 (0.086)
RFC Accuracy 10 fold: 81.2581 (0.103)
XGB Accuracy 10 fold: 80.5376 (0.081)
MLP Accuracy 10 fold: 79.9140 (0.073)


*Scaling data for KNN run*

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
df_model = data.copy()
#Rescaling features age, trestbps, chol, thalach, oldpeak.
scaler = StandardScaler()
features = [['age', 'trestbps', 'chol', 'thalach', 'oldpeak']]
for feature in features:
  df_model[feature] = scaler.fit_transform(df_model[feature])    
#Create x and y variable
X_ = df_model.drop(columns=['target'])
y_ = df_model['target']  
KNN=KNeighborsClassifier()
scores = cross_val_score(KNN, X_, y_, scoring='accuracy', cv=cv, n_jobs=-1)
#scores1 = cross_val_score(KNN, X, y, scoring='accuracy', cv=cv1, n_jobs=-1)
print('KNN Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))
#print('KNN Accuracy Repeated 10 fold: %.3f (%.3f)' % (mean(scores1), std(scores1)))

KNN Accuracy 10 fold: 81.5806 (0.083)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

SVM.fit(x_train, y_train)
print('SVM Score:', SVM.score(x_test,y_test))
RFC.fit(x_train, y_train)
print('RFC Score:', RFC.score(x_test,y_test))
XGB.fit(x_train, y_train)
print('XGB Score:', XGB.score(x_test,y_test))
KNN.fit(x_train, y_train)
print('KNN Score:', KNN.score(x_test,y_test))
MLP = MLPClassifier(random_state=1, max_iter=150).fit(x_train, y_train)
print('MLP Score:', MLP.score(x_test,y_test))


SVM Score: 0.7912087912087912
RFC Score: 0.8131868131868132
XGB Score: 0.7692307692307693
KNN Score: 0.5714285714285714
MLP Score: 0.6923076923076923


#Chi-Sqr

In [ ]:
#Chi-Square Test

import scipy.stats as stats
from scipy.stats import chi2

def chi_sqr(test_column,output_column):
    observed_value = pd.crosstab(test_column,output_column)
    val = stats.chi2_contingency(observed_value)
    expected_value=val[3]
    no_of_rows=len(observed_value.iloc[0:2,0])
    no_of_columns=len(observed_value.iloc[0,0:2])
    dgof=(no_of_rows-1)*(no_of_columns-1)
    alpha=0.05
    for o,e in zip(observed_value.values,expected_value):
        chi_2 = sum([(o-e)**2./e])       
    chi2_stat = np.sum(chi_2)
    return chi2_stat
    
df1 = data.drop(['target'], axis=1)
top_n1 =[]
for i in df1.columns.values:
    top_n1.append(chi_sqr(df1[i],data['target']))
    
col_name1 = np.array(X.columns)
a1 = pd.DataFrame(top_n1)
b1 = pd.DataFrame(col_name1)
info1 = pd.concat([a1,b1], axis=1)
info1.columns = ['Score','Features']

top1 = info1.nsmallest(20,'Score')
ft1 = np.array(top1['Features'])
top1
ft1

array(['fbs', 'chol', 'thalach', 'age', 'trestbps', 'oldpeak', 'ca',
       'restecg', 'cp', 'sex', 'slope', 'exang', 'thal'], dtype=object)

#ReliefF

In [ ]:
pip install skrebate

  Created wheel for skrebate: filename=skrebate-0.61-cp36-none-any.whl size=29259 sha256=7fbbeb2ca7988c9f158cd66ae34fe02b05ab741d4ac17c0649fcd22e7813b57d
  Stored in directory: /root/.cache/pip/wheels/ae/d8/ae/9b51d487e9d02219996d6c260255a216ef07d905b0a0b00ce3
Successfully built skrebate


In [ ]:
#ReliefF

from sklearn.pipeline import make_pipeline
from skrebate import ReliefF

features, classes = data.drop('target', axis=1).values, data['target'].values
X_train, X_test, y_train, y_test = train_test_split(features, classes)

arr = X_train.astype('float64')
fs = ReliefF()
fs.fit(arr, y_train)

top_n2=[]
names=[]
for feature_name, feature_score in zip(data.drop('target', axis=1).columns, fs.feature_importances_):
    top_n2.append(feature_score)
    names.append(feature_name)
    
col_name3 = np.array(data.columns)
a3 = pd.DataFrame(top_n2)
b3 = pd.DataFrame(names)

info3 = pd.concat([a3,b3], axis=1)
info3.columns = ['Score','Features'] 

top3 = info3.nlargest(20,'Score')
ft3 = np.array(top3['Features'])
ft3

array(['thal', 'exang', 'cp', 'ca', 'slope', 'thalach', 'oldpeak', 'sex',
       'restecg', 'age', 'chol', 'fbs', 'trestbps'], dtype=object)

#Mutual Info

In [ ]:
#Information Gain/Mutual Information

from sklearn.feature_selection import mutual_info_classif

res = mutual_info_classif(X,y,discrete_features=True)

col_name2 = np.array(X.columns)

a2 = pd.DataFrame(res)
b2 = pd.DataFrame(col_name2)
info2 = pd.concat([a2,b2], axis=1)
info2.columns = ['Info_Gain','Features']

top2 = info2.nlargest(20,'Info_Gain')
ft2 = np.array(top2['Features'])
ft2

array(['chol', 'thalach', 'oldpeak', 'thal', 'cp', 'ca', 'exang',
       'trestbps', 'age', 'slope', 'sex', 'restecg', 'fbs'], dtype=object)

*Interscetion Check*

In [ ]:
from functools import reduce
def top_fs_int(n):
    f1 = ft1[:n]
    f2 = ft2[:n]
    f3 = ft3[:n]
    return(reduce(np.intersect1d, (f1,f2,f3)))

In [ ]:
top_fs_int(5)

array([], dtype=object)

In [ ]:
top_fs_int(10)

array(['age', 'ca', 'cp', 'oldpeak', 'thalach'], dtype=object)

#Union Check

In [ ]:
def top_fs_union(n):
    f1 = ft1[:n]
    f2 = ft2[:n]
    f3 = ft3[:n]
    return(reduce(np.union1d, (f1,f2,f3)))

#Assemble Run

*Split data*

In [ ]:
def assemble_run(n,classifier,test_size):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['target'], axis=0)
  dfs = data.loc[:,top_fs]
  X = dfs.drop(['target'], axis=1)
  y = dfs['target']
  from sklearn.model_selection import train_test_split
  x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=test_size)
  if classifier==SVC:
      model = SVC(kernel='linear', class_weight='balanced')
      #print ('a')
  else:    
    model = classifier()
  model.fit(x_train, y_train)
  print (model.score(x_test,y_test))

*Cross Validation Run*

In [ ]:
def assemble_run(n,classifier):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['target'], axis=0)
  dfs = data.loc[:,top_fs]
  X = dfs.drop(['target'], axis=1)
  y = dfs['target']
  if classifier==SVC:
      model = SVC(kernel='linear', class_weight='balanced')
  elif classifier==KNeighborsClassifier:
    df_model = dfs.copy()
    #Rescaling features age, trestbps, chol, thalach, oldpeak.
    scaler = StandardScaler()
    features = [['age', 'trestbps', 'chol', 'thalach', 'oldpeak']]
    for feature in features:
      df_model[feature] = scaler.fit_transform(df_model[feature])    
    #Create x and y variable
    X = df_model.drop(columns=['target'])
    y = df_model['target']  
    model = classifier()  
  else:    
    model = classifier()
  cv = KFold(n_splits=10, random_state=1, shuffle=True)
  scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  print('Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))


In [ ]:
#SUPPORT VECTOR MACHINE

#Feature Subset by Union of 5 Best Features from each
SVM1 = assemble_run(5,SVC)
#Feature Subset by Union of 7 Best Features from each
SVM1 = assemble_run(7,SVC)
#Feature Subset by Union of 10 Best Features from each
SVM2 = assemble_run(10,SVC)


Accuracy 10 fold: 82.5591 (0.084)
Accuracy 10 fold: 82.5591 (0.084)
Accuracy 10 fold: 81.8817 (0.086)


In [ ]:
#RANDOM FOREST CLASSIFIER

#Feature Subset by Union of 5 Best Features from each
RFC1 = assemble_run(5,RandomForestClassifier)
#Feature Subset by Union of 7 Best Features from each
RFC2 = assemble_run(7,RandomForestClassifier)
#Feature Subset by Union of 10 Best Features from each
RFC3 = assemble_run(10,RandomForestClassifier)


Accuracy 10 fold: 81.2473 (0.108)
Accuracy 10 fold: 80.5591 (0.102)
Accuracy 10 fold: 82.8925 (0.095)


In [ ]:
#MLP CLASSIFIER

#Feature Subset by Union of 5 Best Features from each
MLP1 = assemble_run(5,MLPClassifier)
#Feature Subset by Union of 7 Best Features from each
MLP2 = assemble_run(7,MLPClassifier)
#Feature Subset by Union of 10 Best Features from each
MLP3 = assemble_run(10,MLPClassifier)


Accuracy 10 fold: 78.5806 (0.077)
Accuracy 10 fold: 79.2581 (0.086)
Accuracy 10 fold: 83.1935 (0.079)


In [ ]:
#XG BOOST CLASSIFIER
#Feature Subset by Union of 5 Best Features from each
XGB1 = assemble_run(5,XGBClassifier)
#Feature Subset by Union of 7 Best Features from each
XGB2 = assemble_run(7,XGBClassifier)
#Feature Subset by Union of 10 Best Features from each
XGB3 = assemble_run(10,XGBClassifier)

Accuracy 10 fold: 79.9032 (0.084)
Accuracy 10 fold: 79.9032 (0.084)
Accuracy 10 fold: 80.5376 (0.081)


In [ ]:
#KNN CLASSIFIER
from sklearn.preprocessing import StandardScaler
#Feature Subset by Union of 5 Best Features from each
KNN1 = assemble_run(5,KNeighborsClassifier)
#Feature Subset by Union of 7 Best Features from each
KNN2 = assemble_run(7,KNeighborsClassifier)
#Feature Subset by Union of 10 Best Features from each
KNN3 = assemble_run(10,KNeighborsClassifier)
KNN3 = assemble_run(13,KNeighborsClassifier)



Accuracy 10 fold: 79.5914 (0.094)
Accuracy 10 fold: 79.5914 (0.094)
Accuracy 10 fold: 81.5806 (0.083)
Accuracy 10 fold: 81.5806 (0.083)


https://medium.com/datadriveninvestor/increase-10-accuracy-with-re-scaling-features-in-k-nearest-neighbors-python-code-677d28032a45#:~:text=From%20what%20we%20have%20done,based%20algorithms%20such%20as%20KNN.
